In [ ]:
import pandas as pd

In [1]:
global_dir = '/Users/bram/Desktop/Honors Draft for Reviewers'

In [3]:
# Get private non financial firm debt 
def read_pfd_data():
    url = "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_TC/2.0/Q.AR+AT+AU+BE+BR+CA+CH+CL+CN+CO+CZ+DE+DK+ES+FI+FR+GB+GR+HK+HU+ID+IE+IL+IN+IT+JP+KR+LU+MX+MY+NL+NO+NZ+PL+PT+RU+SA+SE+SG+TH+TR+US+XM+ZA.N.A.M.770.A?startPeriod=1947-01-01&endPeriod=2024-10-26&format=csv"
    df = pd.read_csv(url)
    df = df[df['TC_LENDERS'] != 'B']  # TC_Lenders the B means bank only and A means all so keep all
    df = df[['BORROWERS_CTY', 'TIME_PERIOD', 'OBS_VALUE']].rename(columns={'BORROWERS_CTY': 'Code', 'OBS_VALUE': 'private_debt'})
    df['TIME_PERIOD'] = (pd.to_datetime(df['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))
    privatedebt = df.set_index(['Code', 'TIME_PERIOD'])
    return privatedebt
private_firm_debt = read_pfd_data()

/var/folders/mg/jjtr9hhj5zj6g0svf83l817h0000gn/T/ipykernel_6382/1672078462.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TIME_PERIOD'] = (pd.to_datetime(df['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))


In [4]:
### Get hhd data
def read_hhd():
    hhd_url = "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_TC/2.0/Q.AR+AT+AU+BE+BR+CA+CH+CL+CN+CO+CZ+DE+DK+ES+FI+FR+GB+GR+HK+HU+ID+IE+IL+IN+IT+JP+KR+LU+MX+MY+NL+NO+NZ+PL+PT+RU+SA+SE+SG+TH+TR+US+XM+ZA.H.A.M.770.A?startPeriod=1947-01-01&endPeriod=2024-10-26&format=csv"
    hhd_df = pd.read_csv(hhd_url)
    hhd_df = hhd_df[['BORROWERS_CTY', 'TIME_PERIOD', 'OBS_VALUE']].rename(columns={'BORROWERS_CTY': 'Code', 'OBS_VALUE': 'household_debt'})
    hhd_df['TIME_PERIOD'] = (pd.to_datetime(hhd_df['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))
    hhd_df = hhd_df.set_index(['Code', 'TIME_PERIOD'])
    return hhd_df
hhd_df = read_hhd()

/var/folders/mg/jjtr9hhj5zj6g0svf83l817h0000gn/T/ipykernel_6382/2946159009.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hhd_df['TIME_PERIOD'] = (pd.to_datetime(hhd_df['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))


In [5]:
hhd_df[hhd_df.index.get_level_values("Code") == "ZA"]

household_debt
Code TIME_PERIOD                
ZA   2008-03-31             44.4
     2008-06-30             43.4
     2008-09-30             42.6
     2008-12-31             41.8
     2009-03-31             41.5
...                          ...
     2023-03-31             34.4
     2023-06-30             34.4
     2023-09-30             34.3
     2023-12-31             34.4
     2024-03-31             34.6

[65 rows x 1 columns]

In [6]:
def read_policy_rate():
    policy_rate_url = "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_CBPOL/1.0/M.AR+AU+BE+BR+CA+CH+CL+CN+CO+CZ+DE+DK+ES+FR+GB+HK+HR+HU+ID+IL+IN+IS+IT+JP+KR+MA+MK+MX+MY+NL+NO+NZ+PE+PH+PL+PT+RO+RS+RU+SA+SE+TH+TR+US+XM+ZA?startPeriod=1944-01-01&endPeriod=2024-10-13&format=csv"
    policy_rate = (pd.read_csv(policy_rate_url)
                   .rename(columns={'OBS_VALUE': 'policy_rate', 'REF_AREA': 'Code'})
                   .loc[:, ['policy_rate', 'Code', 'TIME_PERIOD']]
                   .dropna())
    # Convert TIME_PERIOD to datetime and set as index
    policy_rate['TIME_PERIOD'] = pd.to_datetime(policy_rate['TIME_PERIOD'], format='%Y-%m')
    policy_rate = policy_rate.set_index('TIME_PERIOD').groupby('Code').resample('Q').mean()
    return policy_rate
policyrate = read_policy_rate()
policyrate[policyrate.index.get_level_values("Code") == "ZA"]

policy_rate
Code TIME_PERIOD             
ZA   1980-12-31      7.000000
     1981-03-31      7.666667
     1981-06-30      9.333333
     1981-09-30     12.500000
     1981-12-31     12.833333
...                       ...
     2023-12-31      8.250000
     2024-03-31      8.250000
     2024-06-30      8.250000
     2024-09-30      8.166667
     2024-12-31      8.000000

[177 rows x 1 columns]

In [7]:
def read_gdp():
    gdp = pd.read_excel('/Users/bram/Desktop/Honors Economics/Data and Data Cleaning/RealGDP.xlsx')  
    gdpdf = pd.DataFrame(
        gdp
            .set_index(['Country', 'Scale'])
            .stack()
    ).reset_index().rename(
        columns={'level_2': 'TIME_PERIOD', 0: 'GDP'}
    )
    gdpdf['TIME_PERIOD'] = (pd.to_datetime(gdpdf['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))
    return gdpdf
gdp_df = read_gdp()
gdp_df 

/var/folders/mg/jjtr9hhj5zj6g0svf83l817h0000gn/T/ipykernel_6382/3613256004.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gdpdf['TIME_PERIOD'] = (pd.to_datetime(gdpdf['TIME_PERIOD'].str.replace('-', '')) + pd.offsets.QuarterEnd(0))


,Country,Scale,TIME_PERIOD,GDP
0,Argentina,Millions,1950-03-31,...
1,Argentina,Millions,1950-06-30,...
2,Argentina,Millions,1950-09-30,...
3,Argentina,Millions,1950-12-31,...
4,Argentina,Millions,1951-03-31,...
...,...,...,...,...
19365,United States,Millions,2023-06-30,5556337.5
19366,United States,Millions,2023-09-30,5622673
19367,United States,Millions,2023-12-31,5669814
19368,United States,Millions,2024-03-31,5689688


In [8]:
def read_cpi():
    urls = ["https://stats.bis.org/api/v2/data/dataflow/BIS/WS_LONG_CPI/1.0/M.AE+AR+AT+AU+BE+BG+BR+CA+CH+CL+CN+CO+CY+CZ+DE+DK+DZ+EE+ES+FI+FR+GB+GR+HK+HR+HU+ID+IE+IL+IN+IS+IT+JP+KR+KW+LT+LU+LV+MA+MK+MT+MX+MY+NL+NO+NZ+PE+PH+PL+PT+RO+RS+RU+SA+SE+SG+SI+SK+TH+TR+US+XM+ZA.628?startPeriod=1660-01-01&endPeriod=2024-12-07&format=csv"]
    df = pd.concat([pd.read_csv(url) for url in urls])
    df = df[df['OBS_STATUS']=='A'] # Removes missing values
    df = df[['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE']]
    df = df.rename(columns = {"REF_AREA":"Code", "OBS_VALUE": "CPI"})
    df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], format='%Y-%m')
    df = df.set_index('TIME_PERIOD').groupby('Code').resample('Q').mean()
    return df
cpi_df = read_cpi()

In [9]:
def read_exchange_rate():
    urls = ["https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/M.R?format=csv"]
    df = pd.concat([pd.read_csv(url) for url in urls])
    df = df[['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE']]
    df = df.rename(columns = {"REF_AREA": "Code", "OBS_VALUE": "exchange_rate"})
    df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], format='%Y-%m')
    df['Code'] = df['Code'].astype(str)
    df = df.set_index('TIME_PERIOD')
    df = df.groupby('Code').resample('Q').mean()
    return df
exchange_rate_df = read_exchange_rate()

In [10]:
unique_codes = [
    'NZ', 'DE', 'GR', 'BR', 'TH', 'BE', 'TR', '5A', 'PL', 'NO', 'CZ', 'IT', 'KR', 
    'SG', 'HU', '5R', 'PT', 'IN', 'US', 'NL', 'LU', 'GB', 'ID', 'FI', 'SE', 'CL', 
    'IE', 'ZA', 'SA', 'MX', 'MY', 'FR', 'AR', 'AT', 'IL', 'CH', 'CN', 'ES', '4T', 
    'CO', 'CA', 'AU', 'G2', 'RU', 'DK', 'XM', 'HK', 'JP'
]
code_map = {
    'AL': 'Albania',
    'DZ': 'Algeria',
    'AR': 'Argentina',
    'AM': 'Armenia, Rep. of',
    'AU': 'Australia',
    'AT': 'Austria',
    'AZ': 'Azerbaijan, Rep. of',
    'BS': 'Bahamas, The',
    'BY': 'Belarus, Rep. of',
    'BE': 'Belgium',
    'BO': 'Bolivia',
    'BA': 'Bosnia and Herzegovina',
    'BW': 'Botswana',
    'BR': 'Brazil',
    'BN': 'Brunei Darussalam',
    'BG': 'Bulgaria',
    'CV': 'Cabo Verde',
    'CM': 'Cameroon',
    'CA': 'Canada',
    'CL': 'Chile',
    'HK': 'China, P.R.: Hong Kong',
    'MO': 'China, P.R.: Macao',
    'CN': 'China, P.R.: Mainland',
    'CO': 'Colombia',
    'CR': 'Costa Rica',
    'HR': 'Croatia, Rep. of',
    'CY': 'Cyprus',
    'CZ': 'Czech Rep.',
    'DK': 'Denmark',
    'DO': 'Dominican Rep.',
    'EC': 'Ecuador',
    'EG': 'Egypt, Arab Rep. of',
    'SV': 'El Salvador',
    'EE': 'Estonia, Rep. of',
    'EA': 'Euro Area',
    'FI': 'Finland',
    'FR': 'France',
    'GE': 'Georgia',
    'DE': 'Germany',
    'GH': 'Ghana',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'HU': 'Hungary',
    'IS': 'Iceland',
    'IN': 'India',
    'ID': 'Indonesia',
    'IR': 'Iran, Islamic Rep. of',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JP': 'Japan',
    'JO': 'Jordan',
    'KZ': 'Kazakhstan, Rep. of',
    'KE': 'Kenya',
    'KR': 'Korea, Rep. of',
    'XK': 'Kosovo, Rep. of',
    'KG': 'Kyrgyz Rep.',
    'LV': 'Latvia',
    'LS': 'Lesotho, Kingdom of',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'MY': 'Malaysia',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MX': 'Mexico',
    'MD': 'Moldova, Rep. of',
    'MN': 'Mongolia',
    'ME': 'Montenegro',
    'NA': 'Namibia',
    'NL': 'Netherlands, The',
    'NZ': 'New Zealand',
    'NI': 'Nicaragua',
    'NG': 'Nigeria',
    'MK': 'North Macedonia, Republic of',
    'NO': 'Norway',
    'PY': 'Paraguay',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PL': 'Poland, Rep. of',
    'PT': 'Portugal',
    'QA': 'Qatar',
    'RO': 'Romania',
    'RU': 'Russian Federation',
    'RW': 'Rwanda',
    'WS': 'Samoa',
    'SA': 'Saudi Arabia',
    'SN': 'Senegal',
    'RS': 'Serbia, Rep. of',
    'SC': 'Seychelles',
    'SG': 'Singapore',
    'SK': 'Slovak Rep.',
    'SI': 'Slovenia, Rep. of',
    'ZA': 'South Africa',
    'ES': 'Spain',
    'LK': 'Sri Lanka',
    'LC': 'St. Lucia',
    'VC': 'St. Vincent and the Grenadines',
    'SE': 'Sweden',
    'CH': 'Switzerland',
    'TH': 'Thailand',
    'TT': 'Trinidad and Tobago',
    'TR': 'Türkiye, Rep of',
    'UG': 'Uganda',
    'UA': 'Ukraine',
    'GB': 'United Kingdom',
    'US': 'United States',
    'UY': 'Uruguay',
    'PS': 'West Bank and Gaza'
}

def map_country2codes(unique_codes, code_map):
    # Create a DataFrame from unique items
    df_unique = pd.DataFrame(unique_codes, columns=['Code'])
    # Map the country names based on the codes
    df_unique['Country'] = df_unique['Code'].map(code_map)
    return df_unique
countrycode_df = map_country2codes(unique_codes, code_map)

In [11]:
merged_df = pd.merge(private_firm_debt, hhd_df, on=['Code', 'TIME_PERIOD'], how='inner')
merge_cpi = pd.merge(merged_df, cpi_df, on=['Code', 'TIME_PERIOD'])
merge_er = pd.merge(merge_cpi.reset_index(), exchange_rate_df.reset_index(), on=['Code', 'TIME_PERIOD'])    
new_df = pd.merge(merge_er, countrycode_df, on=['Code'], how='inner')
merge_gdp = pd.merge(new_df, gdp_df, on=['Country', 'TIME_PERIOD'])
mergepr = pd.merge(merge_gdp, policyrate, on=['Code', 'TIME_PERIOD'], how='inner')
mergepr

,Code,TIME_PERIOD,private_debt,household_debt,CPI,exchange_rate,Country,Scale,GDP,policy_rate
0,CO,1996-12-31,31.022,16.534,36.216833,131.660000,Colombia,Millions,...,33.666667
1,CO,1997-03-31,33.207,15.983,38.055133,134.266667,Colombia,Millions,...,31.500000
2,CO,1997-06-30,35.082,15.518,40.050533,138.750000,Colombia,Millions,...,27.833333
3,CO,1997-09-30,37.165,15.550,41.406133,134.413333,Colombia,Millions,...,27.166667
4,CO,1997-12-31,38.785,15.410,42.626833,124.623333,Colombia,Millions,...,27.000000
...,...,...,...,...,...,...,...,...,...,...
3913,BE,1997-12-31,77.300,40.300,78.059284,94.263333,Belgium,Millions,70827.290419,4.550000
3914,BE,1998-03-31,77.600,39.700,78.116702,93.953333,Belgium,Millions,71011.358822,4.550000
3915,BE,1998-06-30,79.000,41.600,78.662169,94.590000,Belgium,Millions,71247.486976,4.550000
3916,BE,1998-09-30,80.600,41.500,78.723175,95.525000,Belgium,Millions,71455.725978,4.533333


In [12]:
def cleandata(finaldf):
    cleaneddf = finaldf.copy()
    cleaneddf.set_index('TIME_PERIOD', inplace=True)
    cleaneddf = cleaneddf[cleaneddf['GDP'] != '...']
    cleaneddf['GDP'] = cleaneddf['GDP'].astype(float)
    cleaneddf['exchange_rate'] = cleaneddf['exchange_rate'].astype(float)
    cleaneddf = cleaneddf.drop(columns ={"Scale"})
    cleaneddf.dropna(subset=['policy_rate'], inplace=True)
    return cleaneddf
# cleaned_df = cleandata(merge_gdp)
cleaned_df = cleandata(mergepr)

In [13]:
def create_excel(df):
    df.to_excel("WithPolicyRate.xlsx", index=True)
create_excel(cleaned_df)